### Lokaler Chatserver/ 

Starten Sie den Server, indem Sie die das Notebook ausführen. Achten Sie darauf, ob Sie unten "Waiting for connection" sehen. Sollte das nicht der Fall sein, stoppen Sie den Kernel und starten erneut. Falls "Address already in use" erscheint, warten Sie etwas ab, bevor Sie neustarten. Alternativ können Sie auch über das "Hub Control Panel" Ihren Server neu starten.

In [1]:
import socket
import threading

In [2]:
host = ('', 5000)
BUFSIZ = 1024
WAITING_LIMIT = 5

In [3]:
# Diese Funktion wird in einem Thread ausgeführt, der beim Start der Anwendung instanziiert wird
def accept_incoming_connections(clients):
    """Sets up handling for incoming clients."""
    while True:
        # Hier warten wir auf eine neue Anmeldung
        client_socket, client_address = server_socket.accept()
        print(f'{client_address} ist beigetreten.')
        # Wir senden ein Eingabeaufforderung
        client_socket.send('Bitte Namen eingeben'.encode())
        # Antwort erwarten
        name = client_socket.recv(BUFSIZ).decode()
        # Wir informieren alle bereits angemeldeten über den Neuzugang
        msg = f'{name} ist im Chat!'
        broadcast(msg, clients)
        print(msg)
        # die weitere Behandlung der Ein- und Ausgaben für den Client überlassen wir einem weiteren Thread
        client_thread = threading.Thread(target=handle_client, args=(clients, name))
        # Wir registrieren den neuen Client mit Thread und Socket
        clients[name] = {'socket': client_socket, 'thread': client_thread}
        # und starten ihn
        client_thread.start()
        # Man beachte, dass man einem Thread Argumente übergeben kann, hier den Socket des Clients
    # Achtung. Diese Funktion (und damit der Thread) terminiert nie!

In [4]:
# Diese Funktion wird in einem Thread ausgeführt, der nach der Anmeldung eines neuen Users instanziiert wird
def handle_client(clients, name):  
    """Handles a single client connection."""
    client_socket = clients[name]['socket']
    # Wir lesen die Antwort auf die in accept gesendete Eingabeaufforderung
    welcome = f'Willkommen {name}! Zum Verlassen "<ende>" eingeben.'
    # und schicken ihm eine freundliche Begrüßung
    client_socket.send(welcome.encode())
    while True:
        # Lesen einer neuen Message
        msg = client_socket.recv(BUFSIZ).decode()
        if msg != '<ende>':
            broadcast(msg, clients, prefix=name)
        else:
            # der Client möchte uns verlassen 
            client_socket.send(f' ... und tschüss {name}'.encode())
            # wir schließen den Socket
            client_socket.close()
            # und löschen den Client aus der Liste
            msg = f'{name} hat den Chat verlassen.'
            broadcast(msg, clients)
            print(msg)
            break

In [5]:
def broadcast(msg, clients, prefix='Server'):  # prefix is for name identification.
    """Broadcasts a message to all the clients."""
    for name in clients:
        if clients[name]['thread'].is_alive():
            clients[name]['socket'].send(f'{prefix}: {msg}'.encode())

In [ ]:
# Dictionary der angemeldeten Clients. Dictionary mit Schlüssel "name" und Wert {'socket':  , 'thread'}
clients = {}

# Instanziieren eines TCP-Sockets
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Binden an den Port 5000
server_socket.bind(host)

if __name__ == "__main__":
    # Wir akzeptieren 5 an der Anmeldung wartende Clients (der sechste bekommt eine Fehlermeldung beim connect
    server_socket.listen(WAITING_LIMIT)
    print("Waiting for connection...")
    # Für die weitere Bearbeitung der Anmeldungen starten wir einen neuen Thread
    accept_thread = threading.Thread(target=accept_incoming_connections, args=(clients,))
    accept_thread.start()
    # der "join" wartet auf die Beendigung des Threads
    accept_thread.join()
    # .. und schließt danach den Socket
    server_socket.close()

Waiting for connection...
('127.0.0.1', 49076) ist beigetreten.
Manfred ist im Chat!
('172.18.0.12', 43980) ist beigetreten.
lw6014s ist im Chat!
('172.18.0.14', 39792) ist beigetreten.
cw1515s ist im Chat!
('172.18.0.15', 55774) ist beigetreten.


Exception in thread Thread-9 (handle_client):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3130/1572328689.py", line 13, in handle_client
  File "/tmp/ipykernel_3130/449501296.py", line 5, in broadcast
BrokenPipeError: [Errno 32] Broken pipe


 ist im Chat!
('172.18.0.15', 56404) ist beigetreten.
Philip ist im Chat!
('172.18.0.19', 52362) ist beigetreten.
cp4802s ist im Chat!
